# Example – Closed Volume

## Introduction

This example aims to introduce more comprehensive ways to define fuel, mixture of fuels and the Air and finite rate reactions, using a simple geometry. The example is divided in 3 seperate Tasks.


## Task 1  
### Non-predefined fuel

Every fire in FDS is defined by `&REAC` namelist. So far we used fuels, which were pre-defined in FDS. The list of these
fuels can be found in Table 14.1. of the User guide. The physical properties of these species are known and do not need to be specified.

Complete the pre-built FDS input file for this task {download}`pentane_fuel.fds` .
* Using Pentane as the fuel. For this purpose, specify the fuel name and formula.
* Specify the values for `SOOT_YIELD`, `CO_YIELD` and `HEAT_OF_COMBUSTION` respectively, 0.01, 0.025 and 48 kJ/kg.
* Add slices for Pentane and Oxygen mass fractions with `PBY=0.0` .
* Plot the `SLCF` data with FDSreader or take a snapshot in Smokeview, nearest to 3s.

### Definition
**Mass/Volume fraction** of a substance within a mixture is the ratio of its mass/volume to the total mass/volume of the mixture. 

$$\small\sf
Mass~fraction:~Y_i = m_i/m_T~~~~~~~~~~~~~~~Volume~fraction:~X_i = v_i/v_T
$$

### Solution

In [2]:
import fdsreader
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
Simulation = fdsreader.Simulation('./non-predefined_fuel/')

time = Simulation.slices[0].get_nearest_timestep(3)

plt.imshow(Simulation.slices[0][0].data[time].T, 
               origin = 'lower', 
               extent = Simulation.slices[0].extent.as_list())
        
plt.title('Pentane')
plt.xlabel("X / m")
plt.ylabel("Z / m")
plt.colorbar(label = "Mass Fraction")
        
plt.savefig("figs/mass_frac_pentane.png")

In [ ]:
plt.imshow(Simulation.slices[1][0].data[time].T, 
               origin = 'lower', 
               extent = Simulation.slices[0].extent.as_list())

plt.title('Oxygen')
plt.xlabel("X / m")
plt.ylabel("Z / m")
plt.colorbar(label = "Mass Fraction")


plt.savefig("figs/mass_frac_oxygen.png")

:::{figure-md}

<img src="figs/mass_frac_pentane.png" width="60%">
<img src="figs/mass_frac_oxygen.png" width="60%">

 `Mass Fraction` of Propane and Oxygen in ($\sf t=3~s$) are shown in the above figures respectively.

:::

## Task 2
### Mixture Fractions

In this task, we try to define the reactants and products of a chemical combustion with its stochiometric coefficients in FDS input file, instead of letting FDS to calculate the coefficients itself.

Given is the propane combustion reaction.

$$\small\sf
C_3H_8 + 5~[O_2 + (0.79/0.21)N_2] → 3 CO_2 + 4 H_2O + 5(0.79/0.21)N_2
$$


* According to the balanced reaction, complete the `VOLUME_FRACTION` of products in the FDS input file {download}`mixture_fraction.fds`. 
* Complete two parameters `SPEC_ID_NU` and `NU` in `&REAC` namelist. Use the User guide for the definitions.
* Define two devices for Pentane and Oxygen `Mass Fraction`. Instead of XB for the coordination, use `DB = 'WHOLE DOMAIN'` to measure the parameter in the whole room.

Now run the Simulation and see the results.

* Simulate the same scenario with initial `MASS FRACTION` = 0.03 and 0.12 for Propane in `&INIT` namelist.
* Compare the mass fraction of Propane and Oxygen as well as the time of combustion for all three simulations in Smokeview. 

:::{figure-md}

<img src="figs/initial_mixture.png" width="60%">

Note that the initial condition is not a homogeneous mixture! This slice shows the `Mass Fraction` of Propane in ($\sf t=0~s$)

:::

### Solution

In [17]:
Sim3 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.03/')
Sim6 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.06/')
Sim12 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.12/')

devc3 = Sim3.devices
devc6 = Sim6.devices
devc12 = Sim12.devices

plt.plot(devc3["Time"].data, devc3['C3H8'].data, label='C3H8 MASS FRACTION, 0.03')
plt.plot(devc6["Time"].data, devc6['C3H8'].data, label='C3H8 MASS FRACTION, 0.06')    
plt.plot(devc12["Time"].data, devc12['C3H8'].data, label='C3H8 MASS FRACTION, 0.12')
    
plt.legend(loc='upper right')
plt.title('C3H8')
plt.xlabel('time / s')
plt.ylabel('Mass Fraction')
plt.grid()

plt.savefig('figs/C3H8_MassFractions.png')
plt.close()

In [18]:
Sim3 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.03/')
Sim6 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.06/')
Sim12 = fdsreader.Simulation('./mixture_fraction/propane_mixfrac_0.12/')

devc3 = Sim3.devices
devc6 = Sim6.devices
devc12 = Sim12.devices

plt.plot(devc3["Time"].data, devc3['O2'].data, label='O2 MASS FRACTION, 0.03')
plt.plot(devc6["Time"].data, devc6['O2'].data, label='O2 MASS FRACTION, 0.06')    
plt.plot(devc12["Time"].data, devc12['O2'].data, label='O2 MASS FRACTION, 0.12')
    
plt.legend(loc='upper right')
plt.title('O2')
plt.xlabel('time / s')
plt.ylabel('Mass Fraction')
plt.grid()

plt.savefig('figs/O2_MassFractions.png')
plt.close()

:::{figure-md}

<img src="figs/C3H8_MassFractions.png" width="50%">
<img src="figs/O2_MassFractions.png" width="50%">

`Mass Fraction` of Propane and Oxygen during the combustion, with three different scenarios for initial mass of Propane.

:::

## Task 3
### Finite rate reaction

By default, FDS uses the infinite reaction rate. On the other hand, FDS can also implement finite-rate reactions using an Arrhenius model. In this task a hot particle ignites the homogen mixture of Propane and Air. At the end of the combustion, all of the reactants will be consumed.

The Arrhenius equation is: 

$$\small\sf
k = A \cdot {e}^{-E_a/RT}
$$

The balanced chemical equation for the combustion of propane is:

$$\small\sf
C_3H_8 + 5 O_2 → 3 CO_2 + 4 H_2O
$$

Using the same geometry as previous tasks, complete this FDS input file {download}`arrhenius_test.fds` .

* What do A, E and N_S stands for in FDS file `&REAC` namelist? For this, check out FDS User guide section 15.3 .
* For each species in the reaction, define a device, to measure the mass fraction during the reaction time in the whole room. 
* Plot the HRR
* Plot all the mass fractions together to see the trend of reactants and products, with FDSreader or Excel.
* Do reactions in FDS follow mass conservation? Plot the total mass of species during the reaction.

### Solution

In [32]:
Simulation = fdsreader.Simulation('./reactionrate_arrhenius/')

plt.plot(Simulation.hrr['Time'].data, Simulation.hrr['HRR'].data)
plt.title('HRR')
plt.xlabel('time / s')
plt.ylabel('heat release rate / kW')
plt.grid()

plt.savefig('figs/arrheniusHRR.png', bbox_inches='tight')
plt.close()

:::{figure-md}

<img src="figs/arrheniusHRR.png" width="50%">

:::

In [29]:
devc = Simulation.devices

plt.plot(devc["Time"].data, devc['O2'].data,   label='O2')
plt.plot(devc["Time"].data, devc['C3H8'].data, label='C3H8')
plt.plot(devc["Time"].data, devc['CO2'].data,  label='CO2')
plt.plot(devc["Time"].data, devc['H2O'].data,  label='H2O')

plt.plot(devc["Time"].data, np.around(devc['O2'].data + 
                                      devc['C3H8'].data + 
                                      devc['CO2'].data + 
                                      devc['H2O'].data,3), label='Total mass')
    
plt.legend(loc='upper right')
plt.xlabel('time / s')
plt.ylabel('Mass Fraction')
plt.grid()

plt.savefig("figs/arrhenius_MassFractions.png")
plt.close()

:::{figure-md}

<img src="figs/arrhenius_MassFractions.png" width="70%">

`Mass Fraction` of all species during the reaction.

:::